# Data Map

In [2]:
from pathlib import Path
import os
import pandas as pd
from glob import glob
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from PIL import Image

In [3]:
folder_path=r"/Users/ahmedmaher/Desktop/kaggle_retina/train"
img_paths=glob(folder_path+r"/*.png")
img_paths
csvfilepath=r"/Users/ahmedmaher/Desktop/kaggle_retina/train.csv"
df = pd.read_csv(csvfilepath)

In [5]:
IMG_SIZE = 128
images = []
labels = []
for idx, row in df.iterrows():
    img_path = os.path.join(folder_path, row["id_code"] + ".png")
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img)
        labels.append(row["diagnosis"])

# Normalizing Images

In [7]:
images = np.array(images, dtype='float32') / 255.0  
labels = np.array(labels)
labels_cat = to_categorical(labels, num_classes=5)

# Data Splitting

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels_cat, test_size=0.2, random_state=42, stratify=labels
)

# Data Balancing

In [11]:
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest"
)
datagen.fit(X_train)

# Building the model

In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)), 
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax') 
])

/Users/ahmedmaher/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=64
)

Epoch 1/64


/Users/ahmedmaher/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - accuracy: 0.5110 - loss: 1.2331 - val_accuracy: 0.6682 - val_loss: 0.9383
Epoch 2/64
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.6554 - loss: 0.9488 - val_accuracy: 0.6886 - val_loss: 0.9377
Epoch 3/64
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.6669 - loss: 0.8899 - val_accuracy: 0.6795 - val_loss: 0.8494
Epoch 4/64
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.6834 - loss: 0.8482 - val_accuracy: 0.7068 - val_loss: 0.8526
Epoch 5/64
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 96ms/step - accuracy: 0.7209 - loss: 0.7934 - val_accuracy: 0.7068 - val_loss: 0.8332
Epoch 6/64
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - accuracy: 0.6988 - loss: 0.8231 - val_accuracy: 0.7159 - val_loss: 0.8165
Epoch 7/64
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 98ms/step - accuracy: 0.7012 - loss: 0.8069 - val_accuracy: 0.7091 - val_loss: 0.8534
Epoch 8/64
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - accuracy: 0.7125 - loss: 0.7959 - val_accuracy: 0.7205 - val_loss: 0

In [16]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_accuracy:.4f}")

14/14 - 0s - 27ms/step - accuracy: 0.7114 - loss: 0.7723

✅ Test Accuracy: 0.7114
